<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/MLM_Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notes
# 1. Figure out MLM in HuggingFace API
# 2. Figure out the mask token id issue

In [ ]:
! pip install -qU transformers datasets sentencepiece

     |████████████████████████████████| 1.2 MB 5.7 MB/s 


In [ ]:
from transformers import T5Tokenizer, T5EncoderModel
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

In [ ]:
dataset = load_dataset("squad")

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_name = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.add_tokens(["<mask>"])
tokenizer.mask_token = "<mask>"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("<mask>")
print(f"Mask Token ID: {tokenizer.mask_token_id}")
model = T5EncoderModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.eval()

params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {params}")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Mask Token ID: 32100


Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.4.layer.1.EncDecAttention.q.weight', 'decoder.block.4.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.1.EncDecAttention.o.weight', 'decoder.block.2.layer.2.layer_norm.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.4.layer.0.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decode

Total parameters: 35316992


In [ ]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                           mlm=True,
                                           return_tensors="pt")

In [10]:
tokens = tokenizer("This is some text" * 10, return_tensors="pt")

In [12]:
input_, label_  = collator.torch_mask_tokens(inputs=tokens["input_ids"])

In [15]:
tokenizer.batch_decode(input_)

['Make <mask> some text <mask> is some textThisjava some lumină <mask> is some textThis is some <mask> This is some textThis is <mask> textThis <mask> some textThis <mask> some text <mask> is some text</s>']

In [24]:
tokenizer.batch_decode(label_, skip_special_tokens=True)
# label_

IndexError: ignored

['This']